In [1]:
import os
import sys
import time
import numpy as np
import psi4
import numpy as np
from tqdm import tqdm


In [2]:
def RL(A, k, mode='rl', learning_rate=0.1, discount=0.99, max_episode=30,
       max_pick=None,silent=False,ex=0):
    """Returns the approximate eigenvalue and row indices for the k-sparse
       approximate solution to the lowest eigenpair of a matrix

    Parameters
    ----------
    A : numpy.ndarray
        The matrix to be approximately diagonalized
    k : int
        The maximum number of rows to retain in k-sparse apprx
    mode: str
        What type of diagonalization
        'rl': use reinforcement learning
        'apsci': a posteriori selected CI (top-k rows from full eigensolution)
        'greedy': greedy selection of top-k rows to include
    learning_rate: float
        Rate of weight update during learning. Choose between (0.0,1.0]
        Only makes sense with 'mode' == 'rl'
    discount : float
        How much to discount future gain in RL.
        Only makes sense with 'mode' == 'rl'
        Close to 0.0 : near-sighted learning
        Close to 1.0 : far-sighted learning
    max_episode: int
        Maximum number of training episodes
        Only makes sense with 'mode' == 'rl'
    max_pick: int
        Maximum number of external rows to consider during exploration
        If None, defaults to maximal space possible (not efficient!)
        Only makes sense with 'mode' == 'rl'

    Returns
    -------
    E_best: float
        Lowest eigenvalue from k-sparse approximation
    s: array_like
        Indices corresponding to rows for k-sparse approx solution of eigen

    """

    NDIM = len(A)

    if max_pick is None:
        max_pick = NDIM - k

    if mode == 'rl':
        # initialize with greedy; random init can be good choice too
        E, idx = RL(A, k, mode='greedy')
        #idx = np.random.choice(range(NDIM),size=k,replace=False) 

        # state vector is 1 if "active" SD, 0 otherwise
        state = np.zeros((NDIM), dtype=np.int64)
        state[idx] = 1  # set topk rows from initialization to "active"

        # initial guess; note argwhere returns a 1D array, but we want 0D
        active = np.argwhere(state == 1).reshape(-1,)
        inactive = np.argwhere(state == 0).reshape(-1,)

        # check we conserve row indices
        assert len(active) == k
        assert len(inactive) == NDIM - k

        E, C = np.linalg.eigh(A[np.ix_(active, active)])
        E0, C0 = E[0], C[:, 0]

        # keep initial guess as our "best" so far
        E_best, state_best = E0, active

        # now populate the weights, renormalizing according to size
        w = np.zeros((NDIM))
        #w = np.random.randn(NDIM)

        w[active] = np.abs(C0)  # from initial guess
        w[active] /= np.linalg.norm(w[active])  # normalize
        #w[active] *= len(active)/NDIM  # scale

        w[inactive] = perturb(A, state)  # PT guess at weights
        w[inactive] /= np.linalg.norm(w[inactive])  # normalize
        #w[inactive] *= len(inactive)/NDIM  # scale

        v = np.zeros_like(w)  # auxilliary weights


        if not silent:
            progress_bar = tqdm(range(max_episode))  # keep track of progress
        else:
            progress_bar = range(max_episode)  

        # outer loop is training episode
        for episode in progress_bar:
            # current status printed out to command line

            if not silent:
                progress_bar.set_description("Best energy: %.6f" % E_best)
                #progress_bar.set_description("Current energy: %.6f" % E0)

            explore_rate = np.exp(-learning_rate*(episode+1))  # can tweak as desired

            # set state to be top-k weights from Q-learning
            state *= 0  # reset state vector

            # sometimes top k, sometimes best state reached so far
            if np.random.rand(1) < 0.75:
                #print("top")
                active = np.argsort(w)[::-1][:k].reshape(-1,)
            else:
                #print("best")
                active = state_best

            state[active] = 1
            inactive = np.argwhere(state == 0).reshape(-1,)

            # get energy from top-k
            E, C = np.linalg.eigh(A[np.ix_(active, active)])
            E0, C0 = E[0], C[:, 0]
            #print(E0)

            # check if top-k is globally optimal, save if it is
            if E0 < E_best:
                E_best = E0
                state_best = active

            # inner loop, partition into active search space
            # I'm using their jargon -- "selected" and "expanded"

            # "selected" are the lowest weighted rows in current selection
            active_idx_sorted = np.argsort(w[active])  # sort asc
            selected = np.arange(len(w))[active][active_idx_sorted]

            # "expanded" are the highest estimated rows outside of curr sele
            cj = perturb(A, state)  # PT guess at weights
            inact_idx_sort = np.argsort(np.abs(cj))[::-1]  # desc
            expanded = np.arange(len(w))[inactive][inact_idx_sort][:max_pick]

            # if we don't replace anything, we can't improve, so we exit
            total_replaced = 0
            for j in range(max_pick):
                assert sum(state) == k  # make sure we have consistent # row
                is_replaced = False
                state[expanded[j]] = 1
                for i in range(k):
                    state[selected[i]] = 0
                    active = np.argwhere(state == 1).reshape(-1,)
                    inactive = np.argwhere(state == 0).reshape(-1,)

                    # check we conserve row indices
                    assert len(active) == k
                    assert len(inactive) == NDIM - k

                    E, C = np.linalg.eigh(A[np.ix_(active, active)])
                    Enew, Cnew = E[0], C[:, 0]
                    if Enew < E0 * (1 - explore_rate*np.random.rand(1)):
                        #print("Update",Enew, E0)
                        is_replaced = True

                        total_replaced += 1

                        # note initial state
                        state1 = np.zeros_like(state)
                        state1[selected] = 1

                        # delete selected[i] from selected, add expanded[j]
                        # put [expanded[j]] to make 0-D array
                        p = selected[i]
                        q = expanded[j]

                        sele1, sele2 = np.split(selected, [i])
                        selected = np.concatenate((sele1,
                                                   sele2[1:],
                                                   [expanded[j]]))

                        # update state
                        state2 = np.zeros_like(state)
                        state2[selected] = 1
                        active = np.argwhere(state2 == 1).reshape(-1,)
                        inactive = np.argwhere(state2 == 0).reshape(-1,)

                        # uncomment to test that selection worked as expected
                        #E,C = np.linalg.eigh(A[np.ix_(selected,selected)])
                        #np.testing.assert_allclose(E[0],Enew)

                        # insert update here
                        # calculate local reward
                        R = E0 - Enew

                        # note if swap was globally optimal
                        if Enew < E_best:
                            E_best = Enew
                            state_best = active

                        E0 = Enew
                        C0 = Cnew

                        # get best possible next move (max Q(s',a'))
                        # a' = (p',q')
                        pp = np.arange(len(w))[active][np.argmin(w[active])]
                        qp = np.arange(len(w))[inactive][np.argmax(w[inactive])]

                        #print(p,q,pp,qp)

                        assert w[pp] == min(w[active])
                        assert w[qp] == max(w[inactive])

                        assert p not in active

                        # normal weight update; delta is TD error
                        delta = R + discount*np.dot(w,f(state2,(pp,qp))) - np.dot(w,f(state1,(p,q)))
                        aux = np.dot(f(state1,(p,q)),v) 

                        # update w
                        w += learning_rate *\
                             (delta*f(state1,(p,q)) - discount*aux*f(state2,(pp,qp)))

                        # update v 
                        beta = np.sqrt(learning_rate) 
                        v += beta*(delta - aux)*f(state1,(p,q))

                        break

                    # "p" from "selected" not selected: reset it
                    state[selected[i]] = 1

                # "q" from "expanded" not selected: reset it
                if is_replaced is False:
                    state[expanded[j]] = 0

            if total_replaced == 0:
                break

        return E_best, state_best
    if mode == 'rle':
        # initialize with greedy; random init can be good choice too
        E, idx = RL(A, k, mode='greedy')
        #idx = np.random.choice(range(NDIM),size=k,replace=False) 

        # state vector is 1 if "active" SD, 0 otherwise
        state = np.zeros((NDIM), dtype=np.int64)
        state[idx] = 1  # set topk rows from initialization to "active"

        # initial guess; note argwhere returns a 1D array, but we want 0D
        active = np.argwhere(state == 1).reshape(-1,)
        inactive = np.argwhere(state == 0).reshape(-1,)

        # check we conserve row indices
        assert len(active) == k
        assert len(inactive) == NDIM - k

        E, C = np.linalg.eigh(A[np.ix_(active, active)])
        E0, C0 = E[ex], C[:, 0]

        # keep initial guess as our "best" so far
        E_best, state_best = E0, active

        # now populate the weights, renormalizing according to size
        w = np.zeros((NDIM))
        #w = np.random.randn(NDIM)

        w[active] = np.abs(C0)  # from initial guess
        w[active] /= np.linalg.norm(w[active])  # normalize
        #w[active] *= len(active)/NDIM  # scale

        w[inactive] = perturb(A, state)  # PT guess at weights
        w[inactive] /= np.linalg.norm(w[inactive])  # normalize
        #w[inactive] *= len(inactive)/NDIM  # scale

        v = np.zeros_like(w)  # auxilliary weights


        if not silent:
            progress_bar = tqdm(range(max_episode))  # keep track of progress
        else:
            progress_bar = range(max_episode)  

        # outer loop is training episode
        for episode in progress_bar:
            # current status printed out to command line

            if not silent:
                progress_bar.set_description("Best energy: %.6f" % E_best)
                #progress_bar.set_description("Current energy: %.6f" % E0)

            explore_rate = np.exp(-learning_rate*(episode+1))  # can tweak as desired

            # set state to be top-k weights from Q-learning
            state *= 0  # reset state vector

            # sometimes top k, sometimes best state reached so far
            if np.random.rand(1) < 0.75:
                #print("top")
                active = np.argsort(w)[::-1][:k].reshape(-1,)
            else:
                #print("best")
                active = state_best

            state[active] = 1
            inactive = np.argwhere(state == 0).reshape(-1,)

            # get energy from top-k
            E, C = np.linalg.eigh(A[np.ix_(active, active)])
            E0, C0 = E[ex], C[:, 0]
            #print(E0)

            # check if top-k is globally optimal, save if it is
            if E0 < E_best:
                E_best = E0
                state_best = active

            # inner loop, partition into active search space
            # I'm using their jargon -- "selected" and "expanded"

            # "selected" are the lowest weighted rows in current selection
            active_idx_sorted = np.argsort(w[active])  # sort asc
            selected = np.arange(len(w))[active][active_idx_sorted]

            # "expanded" are the highest estimated rows outside of curr sele
            cj = perturb(A, state)  # PT guess at weights
            inact_idx_sort = np.argsort(np.abs(cj))[::-1]  # desc
            expanded = np.arange(len(w))[inactive][inact_idx_sort][:max_pick]

            # if we don't replace anything, we can't improve, so we exit
            total_replaced = 0
            for j in range(max_pick):
                assert sum(state) == k  # make sure we have consistent # row
                is_replaced = False
                state[expanded[j]] = 1
                for i in range(k):
                    state[selected[i]] = 0
                    active = np.argwhere(state == 1).reshape(-1,)
                    inactive = np.argwhere(state == 0).reshape(-1,)

                    # check we conserve row indices
                    assert len(active) == k
                    assert len(inactive) == NDIM - k

                    E, C = np.linalg.eigh(A[np.ix_(active, active)])
                    Enew, Cnew = E[ex], C[:, 0]
                    if Enew < E0 * (1 - explore_rate*np.random.rand(1)):
                        #print("Update",Enew, E0)
                        is_replaced = True

                        total_replaced += 1

                        # note initial state
                        state1 = np.zeros_like(state)
                        state1[selected] = 1

                        # delete selected[i] from selected, add expanded[j]
                        # put [expanded[j]] to make 0-D array
                        p = selected[i]
                        q = expanded[j]

                        sele1, sele2 = np.split(selected, [i])
                        selected = np.concatenate((sele1,
                                                   sele2[1:],
                                                   [expanded[j]]))

                        # update state
                        state2 = np.zeros_like(state)
                        state2[selected] = 1
                        active = np.argwhere(state2 == 1).reshape(-1,)
                        inactive = np.argwhere(state2 == 0).reshape(-1,)

                        # uncomment to test that selection worked as expected
                        #E,C = np.linalg.eigh(A[np.ix_(selected,selected)])
                        #np.testing.assert_allclose(E[0],Enew)

                        # insert update here
                        # calculate local reward
                        R = E0 - Enew

                        # note if swap was globally optimal
                        if Enew < E_best:
                            E_best = Enew
                            state_best = active

                        E0 = Enew
                        C0 = Cnew

                        # get best possible next move (max Q(s',a'))
                        # a' = (p',q')
                        pp = np.arange(len(w))[active][np.argmin(w[active])]
                        qp = np.arange(len(w))[inactive][np.argmax(w[inactive])]

                        #print(p,q,pp,qp)

                        assert w[pp] == min(w[active])
                        assert w[qp] == max(w[inactive])

                        assert p not in active

                        # normal weight update; delta is TD error
                        delta = R + discount*np.dot(w,f(state2,(pp,qp))) - np.dot(w,f(state1,(p,q)))
                        aux = np.dot(f(state1,(p,q)),v) 

                        # update w
                        w += learning_rate *\
                             (delta*f(state1,(p,q)) - discount*aux*f(state2,(pp,qp)))

                        # update v 
                        beta = np.sqrt(learning_rate) 
                        v += beta*(delta - aux)*f(state1,(p,q))

                        break

                    # "p" from "selected" not selected: reset it
                    state[selected[i]] = 1

                # "q" from "expanded" not selected: reset it
                if is_replaced is False:
                    state[expanded[j]] = 0

            if total_replaced == 0:
                break

        return E_best, state_best
    if mode == 'rleg':
        # initialize with greedy; random init can be good choice too
        E, idx = RL(A, k, mode='greedy')
        #idx = np.random.choice(range(NDIM),size=k,replace=False) 

        # state vector is 1 if "active" SD, 0 otherwise
        state = np.zeros((NDIM), dtype=np.int64)
        state[idx] = 1  # set topk rows from initialization to "active"

        # initial guess; note argwhere returns a 1D array, but we want 0D
        active = np.argwhere(state == 1).reshape(-1,)
        inactive = np.argwhere(state == 0).reshape(-1,)

        # check we conserve row indices
        assert len(active) == k
        assert len(inactive) == NDIM - k

        E, C = np.linalg.eigh(A[np.ix_(active, active)])
        E0, C0 = E[ex], C[:, 0]
        E0 = 20*E[0]+5*E[1]+2.5*E[2]+1.25*E[3]+0.65*E[4]+0.3*E[5]

        # keep initial guess as our "best" so far
        E_best, state_best = E0, active

        # now populate the weights, renormalizing according to size
        w = np.zeros((NDIM))
        #w = np.random.randn(NDIM)

        w[active] = np.abs(C0)  # from initial guess
        w[active] /= np.linalg.norm(w[active])  # normalize
        #w[active] *= len(active)/NDIM  # scale

        w[inactive] = perturb(A, state)  # PT guess at weights
        w[inactive] /= np.linalg.norm(w[inactive])  # normalize
        #w[inactive] *= len(inactive)/NDIM  # scale

        v = np.zeros_like(w)  # auxilliary weights


        if not silent:
            progress_bar = tqdm(range(max_episode))  # keep track of progress
        else:
            progress_bar = range(max_episode)  

        # outer loop is training episode
        for episode in progress_bar:
            # current status printed out to command line

            if not silent:
                progress_bar.set_description("Best energy: %.6f" % E_best)
                #progress_bar.set_description("Current energy: %.6f" % E0)

            explore_rate = np.exp(-learning_rate*(episode+1))  # can tweak as desired

            # set state to be top-k weights from Q-learning
            state *= 0  # reset state vector

            # sometimes top k, sometimes best state reached so far
            if np.random.rand(1) < 0.75:
                #print("top")
                active = np.argsort(w)[::-1][:k].reshape(-1,)
            else:
                #print("best")
                active = state_best

            state[active] = 1
            inactive = np.argwhere(state == 0).reshape(-1,)

            # get energy from top-k
            E, C = np.linalg.eigh(A[np.ix_(active, active)])
            E0, C0 = E[ex], C[:, 0]
            E0=20*E[0]+5*E[1]+2.5*E[2]+1.25*E[3]+0.65*E[4]+0.3*E[5]
            #print(E0)

            # check if top-k is globally optimal, save if it is
            if E0 < E_best:
                E_best = E0
                state_best = active

            # inner loop, partition into active search space
            # I'm using their jargon -- "selected" and "expanded"

            # "selected" are the lowest weighted rows in current selection
            active_idx_sorted = np.argsort(w[active])  # sort asc
            selected = np.arange(len(w))[active][active_idx_sorted]

            # "expanded" are the highest estimated rows outside of curr sele
            cj = perturb(A, state)  # PT guess at weights
            inact_idx_sort = np.argsort(np.abs(cj))[::-1]  # desc
            expanded = np.arange(len(w))[inactive][inact_idx_sort][:max_pick]

            # if we don't replace anything, we can't improve, so we exit
            total_replaced = 0
            for j in range(max_pick):
                assert sum(state) == k  # make sure we have consistent # row
                is_replaced = False
                state[expanded[j]] = 1
                for i in range(k):
                    state[selected[i]] = 0
                    active = np.argwhere(state == 1).reshape(-1,)
                    inactive = np.argwhere(state == 0).reshape(-1,)

                    # check we conserve row indices
                    assert len(active) == k
                    assert len(inactive) == NDIM - k

                    E, C = np.linalg.eigh(A[np.ix_(active, active)])
                    Enew, Cnew = E[ex], C[:, 0]
                    Enew= 20*E[0]+5*E[1]+2.5*E[2]+1.25*E[3]+0.65*E[4]+0.3*E[5]
                    if Enew < E0 * (1 - explore_rate*np.random.rand(1)):
                        #print("Update",Enew, E0)
                        is_replaced = True

                        total_replaced += 1

                        # note initial state
                        state1 = np.zeros_like(state)
                        state1[selected] = 1

                        # delete selected[i] from selected, add expanded[j]
                        # put [expanded[j]] to make 0-D array
                        p = selected[i]
                        q = expanded[j]

                        sele1, sele2 = np.split(selected, [i])
                        selected = np.concatenate((sele1,
                                                   sele2[1:],
                                                   [expanded[j]]))

                        # update state
                        state2 = np.zeros_like(state)
                        state2[selected] = 1
                        active = np.argwhere(state2 == 1).reshape(-1,)
                        inactive = np.argwhere(state2 == 0).reshape(-1,)

                        # uncomment to test that selection worked as expected
                        #E,C = np.linalg.eigh(A[np.ix_(selected,selected)])
                        #np.testing.assert_allclose(E[0],Enew)

                        # insert update here
                        # calculate local reward
                        R = E0 - Enew

                        # note if swap was globally optimal
                        if Enew < E_best:
                            E_best = Enew
                            state_best = active

                        E0 = Enew
                        C0 = Cnew

                        # get best possible next move (max Q(s',a'))
                        # a' = (p',q')
                        pp = np.arange(len(w))[active][np.argmin(w[active])]
                        qp = np.arange(len(w))[inactive][np.argmax(w[inactive])]

                        #print(p,q,pp,qp)

                        assert w[pp] == min(w[active])
                        assert w[qp] == max(w[inactive])

                        assert p not in active

                        # normal weight update; delta is TD error
                        delta = R + discount*np.dot(w,f(state2,(pp,qp))) - np.dot(w,f(state1,(p,q)))
                        aux = np.dot(f(state1,(p,q)),v) 

                        # update w
                        w += learning_rate *\
                             (delta*f(state1,(p,q)) - discount*aux*f(state2,(pp,qp)))

                        # update v 
                        beta = np.sqrt(learning_rate) 
                        v += beta*(delta - aux)*f(state1,(p,q))

                        break

                    # "p" from "selected" not selected: reset it
                    state[selected[i]] = 1

                # "q" from "expanded" not selected: reset it
                if is_replaced is False:
                    state[expanded[j]] = 0

            if total_replaced == 0:
                break

        return E_best, state_best
    elif mode == 'apsci':

        ## a posteriori selected CI
        ## full matrix diagonalization first, take top-k rows/columns
        E, C = np.linalg.eigh(A)
        idx = np.argpartition(np.abs(C[:, 0]), -k)[-k:]  # top-k eigenvec comps
        #
        ## form top-k submatrix and diagonalize
        A1 = A[np.ix_(idx, idx)]
        E_apsci = np.linalg.eigvalsh(A1)[0]

        return E_apsci, idx

    elif mode == 'greedy':
        # naive greedy selected CI
        state = np.zeros((NDIM))
        for i in range(k):
            if i == 0:
                # to begin, just take arbitrary element (not optimal)
                state[0] = 1
            # form top-k submatrix and diagonalize
            active = np.argwhere(state == 1).reshape(-1,)
            inactive = np.argwhere(state == 0).reshape(-1,)
            A1 = A[np.ix_(active, active)]
            E, C = np.linalg.eigh(A1)
            E0, C0 = E[0], C[:, 0]

            if i < k - 1:
                cj = perturb(A, state)
                cj_idx = np.argmax(np.abs(cj))
                state[np.arange(NDIM)[inactive][cj_idx]] = 1
            else:
                break

        E_greedy = E0
        idx = np.argwhere(state == 1)
        assert len(idx) == k
        return E_greedy, idx

    elif mode == 'full':
        # does full matrix diagonalization ("exact" result)
        return np.linalg.eigvalsh(A)[0], None

def f(state,a):
    active = np.argwhere(state == 1).reshape(-1,) 
    vector = np.zeros_like(state)
    p,q = a
    assert state[p] == 1
    assert state[q] == 0
    vector[active] = 1
    vector[q] =  1
    vector[p] = -1
    return vector/np.linalg.norm((vector))

def perturb(A, state):
    """

    Parameters
    ----------
    A : numpy.ndarray
        The matrix under consideration
    state : array_like
        Vector or list with '1' if index is active or '0' if inactive

    Returns
    -------
    c: array_like (dimension is number of "0's" in "state")
        Approximate magnitude of coefficients in the inactive space as obtained
        from (Epstein-Nesbet) perturbation theory

    """

    active = np.argwhere(state == 1).reshape(-1,)  # active row indices
    inactive = np.argwhere(state == 0).reshape(-1,)  # inactive row indices

    # diagonalize submatrix of active x active dimension
    E, C = np.linalg.eigh(A[np.ix_(active, active)])
    E0, C0 = E[0], C[:, 0]

    # obtain PT approximation to importance of inactive rows (determinants)
    a = np.minimum(1e5, np.abs(np.dot(A[np.ix_(inactive, active)], C0)))
    b = np.maximum(1e-5, np.abs(E0-np.diagonal(A[np.ix_(inactive, inactive)])))
    c = np.true_divide(a, b)
    return c


In [3]:

psi4.core.set_output_file('output.dat', False)

numpy_memory = 14

mol = psi4.geometry("""
O
symmetry c1

""")
#                     H-He 	                Li-Ne                   	Na-Ar 
# cc-pVDZ   	[2s1p] → 5 func. 	    [3s2p1d] →   14 func. 	    [4s3p1d] → 18 func.
# cc-pVTZ   	[3s2p1d] → 14 func. 	[4s3p2d1f] → 30 func. 	    [5s4p2d1f] → 34 func.
# cc-pVQZ   	[4s3p2d1f] → 30 func. 	[5s4p3d2f1g] → 55 func. 	[6s5p3d2f1g] → 59 func.
# aug-cc-pVDZ 	[3s2p] → 9 func. 	    [4s3p2d] → 23 func.     	[5s4p2d] → 27 func.
# aug-cc-pVTZ 	[4s3p2d] → 23 func. 	[5s4p3d2f] → 46 func.   	[6s5p3d2f] → 50 func.
# aug-cc-pVQZ 	[5s4p3d2f] → 46 func. 	[6s5p4d3f2g] → 80 func. 	[7s6p4d3f2g] → 84 func. 
psi4.set_options({'basis': 'cc-pVDZ', 'scf_type': 'pk', 'e_convergence': 1e-8, 'd_convergence': 1e-8})

print('\nStarting SCF and integral build...')

scf_e, wfn = psi4.energy('SCF', return_wfn=True)

C = wfn.Ca()

ndocc = wfn.doccpi()[0]
nmo = wfn.nmo()
nvirt = nmo - ndocc
print(ndocc,nmo,nvirt)




Starting SCF and integral build...
4 14 10


In [4]:
t = time.time()
print("may be energy :",psi4.energy('DETCI'))
print('\nPSI4 cisd cal: %.3f seconds.\n' % (time.time() - t))


may be energy : -74.81298981780854

PSI4 cisd cal: 0.866 seconds.



In [5]:
mints = psi4.core.MintsHelper(wfn.basisset())
H = np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential())


MO = np.asarray(mints.mo_spin_eri(C, C))
Vee = np.asarray(mints.mo_eri(C,C,C,C))

H = np.einsum('uj,vi,uv', C, C, H)
Hone = H.copy()
H = np.repeat(H, 2, axis=0)
H = np.repeat(H, 2, axis=1)

spin_ind = np.arange(H.shape[0], dtype=np.int64) % 2
H *= (spin_ind.reshape(-1, 1) == spin_ind)

In [6]:
from scipy.special import comb
nDet_S = ndocc * nvirt * 2
nDet_D = 2 * comb(ndocc, 2) * comb(nvirt, 2) + ndocc**2 * nvirt**2
nDet = 1 + nDet_S + nDet_D


# nDet = comb(nmo, ndocc)**2

# print("FCI nDet :" ,nDet)


from helper_CI import Determinant, HamiltonianGenerator
from itertools import combinations

print('Generating %d CISD Determinants...' % (nDet))

occList = [i for i in range(ndocc)]

det_ref = Determinant(alphaObtList=occList, betaObtList=occList)

detList = det_ref.generateSingleAndDoubleExcitationsOfDet(nmo)
detList.append(det_ref)

Hamiltonian_generator = HamiltonianGenerator(H, MO)
Hamiltonian_matrixCISD = Hamiltonian_generator.generateMatrix(detList)


Generating 2221 CISD Determinants...


In [7]:
# nDet = comb(nmo, ndocc)**2
# print('Generating %d Full CI Determinants...' % (nDet))
# detList = []
# for alpha in combinations(range(nmo), ndocc):
#     for beta in combinations(range(nmo), ndocc):
#         detList.append(Determinant(alphaObtList=alpha, betaObtList=beta))


# t = time.time()
# Hamiltonian_generator = HamiltonianGenerator(H, MO)
# Hamiltonian_matrixFCI = Hamiltonian_generator.generateMatrix(detList)

In [10]:
e_cisd, wavefunctions = np.linalg.eigh(Hamiltonian_matrixCISD)
# e_fci, wavefunctions = np.linalg.eigh(Hamiltonian_matrixFCI)
# for i in range(0,10):
#         print("Energies ",i,"FCI :% 16.15f"% (e_fci[i]),"CISD   :% 16.15f"% (e_cisd[i]))
        
A= Hamiltonian_matrixCISD
from numpy import count_nonzero
sparsity = len(A)*np.sqrt( count_nonzero(A) / float(A.size) )
print(sparsity)
k = 845  # sparsity
print("k:    ", k)
print("NDet: ", len(A))

E_exact = RL(A, k, mode='full') 
print("Exact:  ", E_exact)

E_apsci, sapsci = RL(A, k, mode='apsci')
print("ap-sCI: ", E_apsci)

E_greedy, sgreedy = RL(A, k, mode='greedy')
print("greedy: ", E_greedy)




843.2514452996804
k:     845
NDet:  2221
Exact:   (-74.84223467819504, None)
ap-sCI:  -74.84223467819504
greedy:  -74.8419853278891


In [11]:
s= np.array([x[0] for x in sgreedy[:]])
# print(s)
E, C = np.linalg.eigh(A[np.ix_(sapsci, sapsci)])
Eg, C = np.linalg.eigh(A[np.ix_(s, s)])
for i in range(0,10):
#         print(i,"FCI :% 16.15f"% (e_fci[i]),"CISD   :% 16.15f"% (e_cisd[i]),"ap-sCI :% 16.15f"% (E[i]),"Greedy :% 16.15f"% (Eg[i]))
    print(i,"CISD   :% 16.15f"% (e_cisd[i]),"ap-sCI :% 16.15f"% (E[i]),"Greedy :% 16.15f"% (Eg[i]))

0 CISD   :-74.842234678195027 ap-sCI :-74.842234678195041 Greedy :-74.841985327889105
1 CISD   :-74.842234678194998 ap-sCI :-74.842234678194984 Greedy :-74.794797992101351
2 CISD   :-74.816507361171588 ap-sCI :-74.807809801454368 Greedy :-74.766544743833563
3 CISD   :-74.812989817820480 ap-sCI :-74.791555107181736 Greedy :-74.725843378829481
4 CISD   :-74.767303682077866 ap-sCI :-74.767303682077781 Greedy :-73.819251036850119
5 CISD   :-74.767303682077767 ap-sCI :-74.767303682077781 Greedy :-73.745208743320816
6 CISD   :-74.735401034467813 ap-sCI :-74.726869687266230 Greedy :-73.726601351715217
7 CISD   :-74.735401034467785 ap-sCI :-74.721743051210197 Greedy :-73.701494181295089
8 CISD   :-74.680531216947230 ap-sCI :-74.660685655751521 Greedy :-73.698767942099380
9 CISD   :-74.238985840820504 ap-sCI :-74.208329717790846 Greedy :-73.638641200711987


In [ ]:
E_rl, sg = RL(A, k, mode='rleg', max_pick=100)
# print("RL2:    ", E_rl)

# E_rle, se1 = RL(A, k, mode='rle', max_pick=None,ex=1)
# E_rle, se2 = RL(A, k, mode='rle', max_pick=None,ex=2)
# E_rle, se3 = RL(A, k, mode='rle', max_pick=None,ex=3)
# E_rle, se4 = RL(A, k, mode='rle', max_pick=None,ex=4)
# E_rle, se5 = RL(A, k, mode='rle', max_pick=None,ex=5)
h

In [ ]:
# print(len(sg),len(se1),len(se2),len(se3),len(se4),len(se5))
# print(se)

# t = np.unique(np.concatenate((sg,se1,se2,se3,se4,se5),0))

In [ ]:
# print(sg)
# print(se)
# print(t)

In [ ]:

# print(len(sg),len(t))
E, C = np.linalg.eigh(A[np.ix_(sg, sg)])
for i in range(0,10):
#         print("Energies ",i,"FCI :% 16.15f"% (e_fci[i]),"CISD   :% 16.15f"% (e_cisd[i]),"RLCI :% 16.15f"% (E[i]))
        print("Energies ",i,"CISD   :% 16.15f"% (e_cisd[i]),"RLCI :% 16.15f"% (E[i]))